In [13]:
import netgen.geom2d as geom2d
from ngsolve import *
from ngsolve.webgui import Draw
from xfem import *
from xfem.lsetcurv import *

geo = geom2d.SplineGeometry()
pnums = [geo.AddPoint (x,y,maxh=0.01) for x,y in [(0,0), (1,0), (1,0.1), (0,0.1)] ]
for p1,p2,bc in [(0,1,"bot"), (1,2,"right"), (2,3,"top"), (3,0,"left")]:
     geo.Append(["line", pnums[p1], pnums[p2]], bc=bc)
mesh = Mesh(geo.GenerateMesh(maxh=0.02))

levelset = 0.04 - sqrt((x-0.5)**2 + (y-0.05)**2)
lsetmeshadap = LevelSetMeshAdaptation(mesh, order=2, threshold=0.1,
                                    discontinuous_qn=True)
deformation = lsetmeshadap.CalcDeformation(levelset)
lsetp1 = lsetmeshadap.lset_p1

#Draw(lsetp1, mesh, "lsetp1")

ci = CutInfo(mesh, lsetp1)
hasneg = ci.GetElementsOfType(HASNEG)

E, nu = 210, 0.2
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

def sigma(u):
    return lam*Trace(Grad(u))*Id(2) + mu*(Grad(u) + (Grad(u)).trans)

factor = Parameter(0)
factor.Set(-0.4)
force = CoefficientFunction( (0,factor) )

fes = H1(mesh, order=4, dirichlet="left", dim=mesh.dim)
u = fes.TrialFunction()
v = fes.TestFunction()

#dx = dCut(lsetp1, NEG, definedonelements=hasneg, deformation=deformation)
dx = dCut(lsetp1, NEG, definedonelements=hasneg) #, deformation=deformation)

a = BilinearForm(fes, symmetric=True, check_unused=False)
a += InnerProduct( sigma(u), Grad(v))*dx

L = LinearForm(fes)
L += force*v*dx

u = GridFunction(fes)

a.Assemble()
L.Assemble()

u.vec.data = a.mat.Inverse(fes.FreeDofs() & GetDofsOfElements(fes, hasneg))*L.vec

#Draw(u, mesh, "def")
#Draw (CF(1), mesh, "C(u)")

scene = DrawDC(lsetmeshadap.lset_p1, CF(1), 0, mesh, "1", deformation=u)

deformation_at_bottom_right = u(mesh(1,0))
print(deformation_at_bottom_right)

(-0.0185939898388516, -0.2796245795227116)
